In [1]:
# import libraries (non-cryptogaphic)
import random # to generate phone numbers
import pandas as pd
from functools import reduce
import os

# import libraries (cryptographic)
import cryptography.hazmat.primitives.asymmetric.dh as dh
import hashlib
import sympy
import secrets

# Generate phone numbers and store them in phone_numbers.csv file

In [2]:
random.seed(10) # to ensure same phone numbers generated every time

In [3]:
# class to generate phone numbers for grab and gojek
class PhoneNumberGenerator:
    def __call__(self, count):
        phone_numbers = random.sample(range(80000000,100000000), count)
        return phone_numbers 

    
# class to store numbers to csv
class PhoneNumberStorageManager:
    def __init__(self):
        self.filename = "phone_numbers.csv"
    def __call__(self, gojek_phone_numbers, grab_phone_numbers):
        d = {"gojek": gojek_phone_numbers, 
            "grab": grab_phone_numbers}
        df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()])) # create dataframe
        df.to_csv(self.filename, index = False)

In [4]:
# generate and store phone numbers
gojek_phone_number_count = 61 # inclusive of phone numbers in common with grab
grab_phone_number_count = 91 # inclusive of phone numbers in common with gojek
common_phone_number_count = 10

# instantiate required classes
phone_number_generator = PhoneNumberGenerator()
phone_number_storage_manager = PhoneNumberStorageManager()

# generate phone numbers
phone_numbers = phone_number_generator(gojek_phone_number_count+grab_phone_number_count-common_phone_number_count)
common_phone_numbers = phone_numbers[0:common_phone_number_count]
gojek_phone_numbers = phone_numbers[0:gojek_phone_number_count]
grab_phone_numbers = common_phone_numbers + phone_numbers[gojek_phone_number_count:] 

#shuffle phone number lists
random.shuffle(gojek_phone_numbers)
random.shuffle(grab_phone_numbers)

# write phone numbers to csv file
phone_number_storage_manager(gojek_phone_numbers, grab_phone_numbers)

# Define classes for the necessary for the algorithm

In [5]:
# class to generate clients' private secret
class NumberGenerator:
    
    def generate_public_parameters(self, size):
        p = self.generate_safe_prime(size)
        print(f"p is prime: {sympy.ntheory.isprime(p)}")
        length_of_p = len(bin(p)[2:]) # should be 1024
        print(f"Length of prime modulus, p: {length_of_p}.\nNote: Should be {size}.")
        q = (p-1)//2
        print(f"q is prime: {sympy.ntheory.isprime(q)}")
        factors_pminus1 = [1, 2, q]
        
        return p, factors_pminus1

    def generate_safe_prime(self, size):
        candidate = dh.generate_parameters(2, size).parameter_numbers().p # generate 1024-bit prime number
        while True:
            is_safe_prime = sympy.ntheory.isprime((candidate-1)//2) # if safe prime, (candidate-1)/2 is prime
            if (is_safe_prime):
                break
            else:
                candidate = dh.generate_parameters(2, size).parameter_numbers().p
                print(candidate)
 
        return candidate
    
    def generate_random_number(self, size):
        return secrets.randbits(size)
            
# class to inspect values          
class NumberInspector:
    
    def check_is_primitive_generator(self, candidate, factors_divisorminus1, divisor): # note: factors should be the factors of divisor-1
        
        # apply lagrange theorem
        for possible_order in factors_divisorminus1: # check congruence for all factors (factors is exclusive of p-1 itself)
            result = pow(candidate, possible_order, divisor) # fast modular exponentiation
            if (result == 1):
                return False
            
        return True 

    
class StorageManager:
    
    def store_data(self, filename, data):
        df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe
        df.to_csv(filename, index=False)

                


    
        

# Create psi client class

In [6]:
# client class (both grab and gojek are clients communicating directly with each other)
class Client:
    def __init__(self, name, private_key_size, phone_numbers, p, factors_pminus1):
        
        self.number_inspector = NumberInspector()
        self.number_generator = NumberGenerator()
        
        self.private_key = self.number_generator.generate_random_number(private_key_size)
        self.my_set = phone_numbers
        self.p = p
        self.factors_pminus1 = factors_pminus1
        
        self.my_hashed_set = None
        self.my_self_encrypted_set = None
        self.my_encrypted_set = None
        self.other_party_encrypted_set = None
        self.common_values = None
        
        # create datafile for communication with another party
        # only need to share self_encrypted_values and other_party_encrypted_values
        self.my_dict = {
            'my_self_encrypted_set': None, 
            'other_party_encrypted_set': None,
            'common_values': None
        }
        
        self.name = name
        self.filename = name + "_data.csv"
        
        self.storage_manager = StorageManager()
        self.storage_manager.store_data(self.filename, self.my_dict)

    def hash_to_primitive_root_modulo_p(self, element):     
        endian = "big"
        element = element.to_bytes(4, endian)
        hash_hex = hashlib.sha256(element).hexdigest() #sha3_256
        hash_int = int(hash_hex, 16)
        while True:
            is_primitive_generator = self.number_inspector.check_is_primitive_generator(
                hash_int, self.factors_pminus1, self.p
            )
            if (is_primitive_generator):
                break
            else:
                hash_int = hash_int.to_bytes(32, endian)
                hash_hex = hashlib.sha256(hash_int).hexdigest()
                hash_int = int(hash_hex, 16)
        return hash_int
    
    def modular_exponentation(self, element):
        return pow(element, self.private_key, self.p)
    
    def hash_set(self):
        
        self.my_hashed_set = []
        
        for element in self.my_set:
            hashed_value = self.hash_to_primitive_root_modulo_p(element)
            self.my_hashed_set.append(hashed_value)
            
    def encrypt_set(self, is_other_party, other_party_set = None):
        
        if (is_other_party):
            other_party_set_int = []
            for element_string in other_party_set:
                other_party_set_int.append(int(element_string))
            set_to_encrypt = other_party_set_int
        else:
            set_to_encrypt = self.my_hashed_set
        
        encrypted_values = []
        for element in set_to_encrypt:
            encrypted_value = self.modular_exponentation(element)
            encrypted_values.append(encrypted_value)
            
        if (is_other_party):
            self.other_party_encrypted_set = encrypted_values
            self.my_dict["other_party_encrypted_set"] = self.other_party_encrypted_set
        else:
            self.my_self_encrypted_set = encrypted_values
            self.my_dict["my_self_encrypted_set"] = self.my_self_encrypted_set
            
        self.storage_manager.store_data(self.filename, self.my_dict)
        
        
    def get_intersection(self, my_encrypted_set):
        
        my_encrypted_set_int = []
        for element_string in my_encrypted_set:
            my_encrypted_set_int.append(int(element_string))
        self.my_encrypted_set = my_encrypted_set_int
        encrypted_common_values = set(self.my_encrypted_set).intersection(self.other_party_encrypted_set)
        index_of_common_values = []
        
        for element in encrypted_common_values:
            index_of_common_values.append(self.my_encrypted_set.index(element))
            
        self.common_values = []
        
        for index in index_of_common_values:
            self.common_values.append(self.my_set[index])
            
        self.my_dict["common_values"] = self.common_values
        self.storage_manager.store_data(self.filename, self.my_dict)
        return self.other_party_encrypted_set, self.my_encrypted_set

    
    def get_stored_data(self):
        return pd.read_csv(self.filename)
    
    def get_dictionary(self):
        return self.my_dict

# Initialize context

In [7]:
# assign pre-determined variables for psi
key_size = 1024 # both private keys and large prime

number_generator = NumberGenerator()
p, factors_pminus1 = number_generator.generate_public_parameters(key_size)

# create clients
grab = Client("grab", key_size, grab_phone_numbers, p, factors_pminus1)
gojek = Client("gojek", key_size, gojek_phone_numbers, p, factors_pminus1)



p is prime: True
Length of prime modulus, p: 1024.
Note: Should be 1024.
q is prime: True


C:\Users\jiaji\AppData\Local\Temp/ipykernel_17340/525289407.py:47: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


# Get intersection

In [8]:
# clients hash their own set
grab.hash_set()
gojek.hash_set()

# clients self encrypt hashed set
grab.encrypt_set(False) # set is_other_party to false to encrypt own hashed set
gojek.encrypt_set(False)

# read both clients self encrypted set
grab_data = grab.get_stored_data()
gojek_data = gojek.get_stored_data()

grab_self_encrypted_set = grab_data["my_self_encrypted_set"].tolist()
gojek_self_encrypted_set = gojek_data["my_self_encrypted_set"].tolist()

# clients encrypt other party's self encrypted set (data exchange occurs here)
grab.encrypt_set(True, gojek_self_encrypted_set) # set is_other_party to true to encrypt the set passed into function
gojek.encrypt_set(True, grab_self_encrypted_set)

# read both clients encrypted set
grab_data = grab.get_stored_data()
gojek_data = gojek.get_stored_data()

gojek_encrypted_set = grab_data["other_party_encrypted_set"].tolist()
grab_encrypted_set = gojek_data["other_party_encrypted_set"].tolist()

# clients find intersection (data exchange occurs here)
gojek_encrypted_set, grab_encrypted_set= grab.get_intersection(grab_encrypted_set)
a,b = gojek.get_intersection(gojek_encrypted_set)

C:\Users\jiaji\AppData\Local\Temp/ipykernel_17340/525289407.py:47: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


# Check results

In [31]:
# get intersection found by the two parties
grab_data = grab.get_stored_data()
gojek_data = gojek.get_stored_data()
gojek_found_intersection = gojek_data["common_values"].tolist()
grab_found_intersection = grab_data["common_values"].tolist()

# sort numbers for easier comparison
gojek_found_intersection.sort()
grab_found_intersection.sort()
common_phone_numbers.sort()

# summarize them in a dataframe
d = {"actual": common_phone_numbers,
    "gojek": gojek_found_intersection,
    "grab": grab_found_intersection}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()])).dropna() 
df

,actual,gojek,grab
0,80497694.0,80497694.0,80497694.0
1,81093373.0,81093373.0,81093373.0
2,86915509.0,86915509.0,86915509.0
3,89312048.0,89312048.0,89312048.0
4,94391128.0,94391128.0,94391128.0
5,95521626.0,95521626.0,95521626.0
6,96192082.0,96192082.0,96192082.0
7,96485172.0,96485172.0,96485172.0
8,99173089.0,99173089.0,99173089.0
9,99397525.0,99397525.0,99397525.0
